In [1]:
# Data Preprocess

In [1]:
# import libraries
import pandas as pd 

import geopandas as gp

from geodatasets import get_path
import numpy as np
import pathlib

from datetime import datetime, timedelta
import folium
import geohash

curr_dir = pathlib.Path().absolute()

DATA_FOLDER = curr_dir.parent.joinpath('data/')

SAVE_FOLDER = DATA_FOLDER.joinpath('preprocessed/')


print(f'DATA_FOLDER = {DATA_FOLDER}')
print(f'SAVE_FOLDER = {SAVE_FOLDER}')




DATA_FOLDER = /Users/ball/Documents/workspace/TGNet/data
SAVE_FOLDER = /Users/ball/Documents/workspace/TGNet/data/preprocessed


In [2]:
# load the data from data/ directory
fields = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']

taxi_data_2015_01_filename = 'yellow_tripdata_2015-01.csv'
taxi_data_2015_01_filename = DATA_FOLDER.joinpath(taxi_data_2015_01_filename)
taxi_data_2015_01 = pd.read_csv(taxi_data_2015_01_filename, usecols=fields, engine="c")
# for testing, set nrows for speed
# taxi_data_2015_01 = pd.read_csv(taxi_data_2015_01_filename, nrows=100000, usecols=fields, engine="c")
print(taxi_data_2015_01.describe())
print(taxi_data_2015_01.head(2))

       passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
count     1.274899e+07      1.274899e+07     1.274899e+07       1.274899e+07   
mean      1.681491e+00     -7.256184e+01     3.997282e+01      -7.260904e+01   
std       1.337924e+00      1.012510e+01     5.578691e+00       9.966037e+00   
min       0.000000e+00     -1.219258e+02     0.000000e+00      -7.401667e+02   
25%       1.000000e+00     -7.399168e+01     4.073554e+01      -7.399120e+01   
50%       1.000000e+00     -7.398160e+01     4.075314e+01      -7.397976e+01   
75%       2.000000e+00     -7.396662e+01     4.076757e+01      -7.396246e+01   
max       9.000000e+00      7.866265e+01     4.047000e+02       8.527402e+01   

       dropoff_latitude  
count      1.274899e+07  
mean       3.999961e+01  
std        5.487742e+00  
min       -9.029157e+00  
25%        4.073436e+01  
50%        4.075362e+01  
75%        4.076880e+01  
max        4.595333e+02  
  tpep_pickup_datetime tpep_dropoff_datetime 

In [3]:
# remove rows that passenger_count is <= 0 or >= 5
print(taxi_data_2015_01.passenger_count.describe())
taxi_data_2015_01 = taxi_data_2015_01[(taxi_data_2015_01.passenger_count >= 1) | (taxi_data_2015_01.passenger_count <= 4)]

taxi_data_2015_01 = taxi_data_2015_01.drop(columns=['passenger_count'])
print(taxi_data_2015_01.head(1))

print(taxi_data_2015_01.head(3))

count    1.274899e+07
mean     1.681491e+00
std      1.337924e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      9.000000e+00
Name: passenger_count, dtype: float64
  tpep_pickup_datetime tpep_dropoff_datetime  pickup_longitude  \
0  2015-01-15 19:05:39   2015-01-15 19:23:42        -73.993896   

   pickup_latitude  dropoff_longitude  dropoff_latitude  
0        40.750111         -73.974785         40.750618  
  tpep_pickup_datetime tpep_dropoff_datetime  pickup_longitude  \
0  2015-01-15 19:05:39   2015-01-15 19:23:42        -73.993896   
1  2015-01-10 20:33:38   2015-01-10 20:53:28        -74.001648   
2  2015-01-10 20:33:38   2015-01-10 20:43:41        -73.963341   

   pickup_latitude  dropoff_longitude  dropoff_latitude  
0        40.750111         -73.974785         40.750618  
1        40.724243         -73.994415         40.759109  
2        40.802788         -73.951820         40.824413  


In [4]:
def filter_pickup_dropoff_time(row):
    pickup_time = row['tpep_pickup_datetime']
    dropoff_time = row['tpep_dropoff_datetime']
    
    pickup_time_obj = datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S')
    dropoff_time_obj = datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S')
    if pickup_time_obj >= dropoff_time_obj:
        # invalid time
        row['filter']=False
    elif dropoff_time_obj - pickup_time_obj > timedelta(hours=1):
        row['filter']=False
    else: 
        row['filter']=True
    return row
    


# remove rows that gap between tpep_pickup_datetime - tpep_dropoff_datetime is neg or more than 1 hours
taxi_data_2015_01 = taxi_data_2015_01.apply(filter_pickup_dropoff_time, axis=1)
taxi_data_2015_01 = taxi_data_2015_01[taxi_data_2015_01['filter']].drop(columns=['filter', 'tpep_dropoff_datetime'])
print(taxi_data_2015_01.head(1))
# remove rows that dates are not in year 2015

KeyboardInterrupt: 

In [ ]:
# process the date part
# change tpep_pickup_datetime 2015-01-15 19:05:39 into 2015-01-15-19-0
taxi_data_2015_01_datetime = taxi_data_2015_01.loc[:, 'tpep_pickup_datetime']

def convert_datetime(datetime_string):
    datetime_obj = datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S')

    # if minute is 0~29, then set minute = 0
    # else, set minute=30
    if datetime_obj.minute >= 30:
        minute = 30 
    else:
        minute = 0
    return datetime_obj.replace(minute=minute).strftime('%Y-%m-%d-%H-%M')
    

taxi_data_2015_01_datetime = taxi_data_2015_01_datetime.apply(convert_datetime)
taxi_data_2015_01_datetime = taxi_data_2015_01_datetime.rename("pickup_datetime")

# print(taxi_data_2015_01_datetime)

# append it to taxi_data_2015_01
taxi_data_2015_01 = taxi_data_2015_01.join(taxi_data_2015_01_datetime)

taxi_data_2015_01 = taxi_data_2015_01.drop('tpep_pickup_datetime', axis=1)
print(taxi_data_2015_01)


In [ ]:
# filter if there are any invalid longitude, latitude
# NYC long range: -74.3 ~ -73.8
# NYC lat range: 40.4 ~ 41
taxi_data_2015_01 = taxi_data_2015_01[(taxi_data_2015_01.pickup_longitude >= -74.3) & (taxi_data_2015_01.pickup_longitude <= -73.6)]

taxi_data_2015_01 = taxi_data_2015_01[(taxi_data_2015_01.dropoff_longitude >= -74.1) & (taxi_data_2015_01.dropoff_longitude <= -73.5)]

taxi_data_2015_01 = taxi_data_2015_01[(taxi_data_2015_01.pickup_latitude >= 40.4) & (taxi_data_2015_01.pickup_latitude <= 41)]

taxi_data_2015_01 = taxi_data_2015_01[(taxi_data_2015_01.dropoff_latitude >= 40.3) & (taxi_data_2015_01.dropoff_latitude <= 41.1)]

# check the max, min longitude and latitude
print(f'Max pickup longitude: {taxi_data_2015_01.pickup_longitude.max()}')
print(f'Min pickup longitude: {taxi_data_2015_01.pickup_longitude.min()}')

print(f'Max pickup latitude: {taxi_data_2015_01.pickup_latitude.max()}')
print(f'Min pickup latitude: {taxi_data_2015_01.pickup_latitude.min()}')

In [ ]:
# convert long, lat into geohash
def convert_geohash(row):
    pickup_lat, pickup_lng = row['pickup_latitude'], row['pickup_longitude']
    dropoff_lat, dropoff_lng = row['dropoff_latitude'], row['dropoff_longitude']

    row['pickup_geohash'] = geohash.encode(latitude=pickup_lat, longitude=pickup_lng, precision=10)
    row['dropoff_geohash'] = geohash.encode(latitude=dropoff_lat, longitude=dropoff_lng, precision=10)
    return row 


taxi_data_2015_01 = taxi_data_2015_01.apply(convert_geohash, axis=1)
taxi_data_2015_01 = taxi_data_2015_01.drop(columns=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])

print(taxi_data_2015_01.head(3))

In [ ]:
# Save the preprocessed file
# for fast read speed, we will store as parquet format
file_name = 'taxi_data_2015_01_preprocessed.parquet.gzip'
file_path = SAVE_FOLDER.joinpath(file_name)
file_path.parent.mkdir(parents=True, exist_ok=True)

taxi_data_2015_01.to_parquet(file_path, compression='gzip')